In [1]:
import numpy as np
import pandas as pd
import seaborn.apionly as sns
import matplotlib.pyplot as plt
sns.set()
color = sns.color_palette()

In [2]:
#setup para el notebook

%matplotlib inline
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (16, 12)

In [3]:
data_dir = "C:\\Users\Rafael\\Documents\\data\\instacart\\raw"
orders = pd.read_csv(data_dir + "/orders.csv", index_col="order_id")

In [4]:
orders

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,
2539329,1,prior,1,2,8,nan
2398795,1,prior,2,3,7,15.00
473747,1,prior,3,3,12,21.00
2254736,1,prior,4,4,7,29.00
431534,1,prior,5,4,15,28.00
...,...,...,...,...,...,...
2266710,206209,prior,10,5,18,29.00
1854736,206209,prior,11,4,10,30.00
626363,206209,prior,12,1,12,18.00


In [5]:
valid_users = orders[orders.eval_set == "train"].user_id.sample(frac=0.1)
valid_users

order_id
1755578      8728
2204031     10816
2907616    111702
1541179     90693
3253474     47593
            ...  
140667      66469
86977      178892
874546      48854
1211659    156103
989471     205903
Name: user_id, dtype: int64

In [6]:
train = pd.read_csv(data_dir + "/order_products__train.csv")
train = train.join(orders.user_id, on="order_id")
train = train[train.user_id.isin(valid_users)]
train = train.join(orders.order_number)

In [7]:
prior = pd.read_csv(data_dir + "/order_products__prior.csv")
prior =  prior.join(orders[["user_id", "order_number"]], on="order_id")
prior = prior[prior.user_id.isin(valid_users)]
prior = prior.join(orders.order_number)

In [8]:
train

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
185,774,47482,1,0,27650,25
186,774,43335,2,1,27650,25
187,774,16108,3,1,27650,25
295,1120,47211,1,1,119791,9
296,1120,45685,2,1,119791,9
...,...,...,...,...,...,...
1384586,3421026,15261,2,0,174668,4
1384587,3421026,32237,3,0,174668,4
1384588,3421026,10,4,0,174668,4
1384589,3421026,4493,5,0,174668,4


In [9]:
prior

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
172,19,42265,1,1,83009,7
173,19,24838,2,1,83009,7
174,19,41540,3,1,83009,7
175,20,35430,1,0,182912,1
176,20,47485,2,0,182912,1
...,...,...,...,...,...,...
32434415,3421072,38275,8,1,50050,3
32434416,3421072,25647,9,1,50050,3
32434417,3421072,20144,10,1,50050,3
32434418,3421072,6473,11,0,50050,3


In [10]:
valid_users.sort_values()

order_id
525192          7
880375          8
2180313        17
2461523        18
1864787        43
            ...  
278249     206150
674892     206156
1620607    206174
36236      206177
1716008    206205
Name: user_id, dtype: int64

In [38]:
nusers = train.user_id.max()
#nusers = 100

In [39]:
def get_real(g): 
    x = g[g.reordered==1]
    x = set(x.product_id.values)
    if len(x) == 0:
        x = set([0])
    return x

In [40]:
real = train[train.user_id <= nusers].groupby("user_id").apply(get_real)
real

user_id
7         {17638, 29894, 47272, 45066, 13198, 37999, 408...
8                              {15937, 41540, 23165, 21903}
17                                    {12720, 16797, 18534}
18                      {25997, 22031, 21137, 36216, 47546}
43        {11777, 33027, 23909, 46981, 29223, 45066, 196...
                                ...                        
206150                         {38928, 22362, 42500, 17957}
206156                  {42820, 20298, 33452, 34483, 15999}
206174    {18370, 5922, 17191, 21137, 12211, 35221, 2392...
206177                    {46979, 21903, 10132, 24852, 311}
206205           {27845, 10181, 28745, 31404, 21137, 22035}
dtype: object

In [41]:
def get_preds(g): 
    x = g[(g.order_number == g.order_number.max())]
    x = set(x.product_id.values)
    if len(x) == 0:
        x = set([0])
    return x

In [42]:
preds = prior[prior.user_id <= nusers].groupby("user_id").apply(get_preds)
preds

user_id
7         {37602, 31683, 47272, 29993, 27690, 15592, 131...
8         {15937, 17794, 8193, 41540, 18531, 15143, 3364...
17        {14146, 48933, 18534, 18567, 9006, 26767, 7350...
18                                     {27729, 2826, 39013}
43        {1667, 22117, 48645, 29223, 10246, 48205, 3678...
                                ...                        
206150                                       {38928, 23915}
206156                          {14612, 5460, 45061, 40910}
206174    {45570, 40706, 24964, 46979, 6287, 13083, 1257...
206177    {24838, 5479, 15399, 47209, 15532, 21903, 3240...
206205     {27845, 19173, 21137, 3896, 22035, 38739, 43352}
dtype: object

In [43]:
preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
preds

,real,pred
user_id,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131..."
8,"{15937, 41540, 23165, 21903}","{15937, 17794, 8193, 41540, 18531, 15143, 3364..."
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 18567, 9006, 26767, 7350..."
18,"{25997, 22031, 21137, 36216, 47546}","{27729, 2826, 39013}"
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{1667, 22117, 48645, 29223, 10246, 48205, 3678..."
...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}"
206156,"{42820, 20298, 33452, 34483, 15999}","{14612, 5460, 45061, 40910}"
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{45570, 40706, 24964, 46979, 6287, 13083, 1257..."


In [44]:
def get_true_pos(g):
    x = g["real"].intersection(g["pred"])
    return x

In [45]:
preds["true_pos"] = preds.apply(get_true_pos, axis=1)
preds

,real,pred,true_pos
user_id,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131...","{47272, 40852, 13198}"
8,"{15937, 41540, 23165, 21903}","{15937, 17794, 8193, 41540, 18531, 15143, 3364...","{15937, 41540, 23165}"
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 18567, 9006, 26767, 7350...","{16797, 18534}"
18,"{25997, 22031, 21137, 36216, 47546}","{27729, 2826, 39013}",{}
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{1667, 22117, 48645, 29223, 10246, 48205, 3678...","{24852, 48205, 29223}"
...,...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}",{38928}
206156,"{42820, 20298, 33452, 34483, 15999}","{14612, 5460, 45061, 40910}",{}
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{45570, 40706, 24964, 46979, 6287, 13083, 1257...",{23926}


In [46]:
preds["prec"] = preds.apply(lambda g: len(g.true_pos) / len(g.pred), axis=1)
preds

,real,pred,true_pos,prec
user_id,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131...","{47272, 40852, 13198}",0.25
8,"{15937, 41540, 23165, 21903}","{15937, 17794, 8193, 41540, 18531, 15143, 3364...","{15937, 41540, 23165}",0.23
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 18567, 9006, 26767, 7350...","{16797, 18534}",0.25
18,"{25997, 22031, 21137, 36216, 47546}","{27729, 2826, 39013}",{},0.00
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{1667, 22117, 48645, 29223, 10246, 48205, 3678...","{24852, 48205, 29223}",0.21
...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}",{38928},0.50
206156,"{42820, 20298, 33452, 34483, 15999}","{14612, 5460, 45061, 40910}",{},0.00
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{45570, 40706, 24964, 46979, 6287, 13083, 1257...",{23926},0.04


In [47]:
preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
preds

,real,pred,true_pos,prec,recall
user_id,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131...","{47272, 40852, 13198}",0.25,0.38
8,"{15937, 41540, 23165, 21903}","{15937, 17794, 8193, 41540, 18531, 15143, 3364...","{15937, 41540, 23165}",0.23,0.75
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 18567, 9006, 26767, 7350...","{16797, 18534}",0.25,0.67
18,"{25997, 22031, 21137, 36216, 47546}","{27729, 2826, 39013}",{},0.00,0.00
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{1667, 22117, 48645, 29223, 10246, 48205, 3678...","{24852, 48205, 29223}",0.21,0.15
...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}",{38928},0.50,0.25
206156,"{42820, 20298, 33452, 34483, 15999}","{14612, 5460, 45061, 40910}",{},0.00,0.00
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{45570, 40706, 24964, 46979, 6287, 13083, 1257...",{23926},0.04,0.12


In [48]:
sorted(preds.loc[7, "real"])

[13198, 17638, 29894, 37999, 40852, 43967, 45066, 47272]

In [49]:
sorted(preds.loc[7, "pred"])

[9598,
 13198,
 15592,
 21137,
 27690,
 29993,
 30391,
 31683,
 32177,
 37602,
 40852,
 47272]

In [50]:
sorted(preds.loc[7, "true_pos"])

[13198, 40852, 47272]

In [51]:
preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
preds

,real,pred,true_pos,prec,recall,f1
user_id,,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131...","{47272, 40852, 13198}",0.25,0.38,0.30
8,"{15937, 41540, 23165, 21903}","{15937, 17794, 8193, 41540, 18531, 15143, 3364...","{15937, 41540, 23165}",0.23,0.75,0.35
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 18567, 9006, 26767, 7350...","{16797, 18534}",0.25,0.67,0.36
18,"{25997, 22031, 21137, 36216, 47546}","{27729, 2826, 39013}",{},0.00,0.00,nan
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{1667, 22117, 48645, 29223, 10246, 48205, 3678...","{24852, 48205, 29223}",0.21,0.15,0.18
...,...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}",{38928},0.50,0.25,0.33
206156,"{42820, 20298, 33452, 34483, 15999}","{14612, 5460, 45061, 40910}",{},0.00,0.00,nan
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{45570, 40706, 24964, 46979, 6287, 13083, 1257...",{23926},0.04,0.12,0.06


In [52]:
preds["f1"] = preds["f1"].fillna(0)

In [53]:
print("resultado final:", preds.f1.mean())

resultado final: 0.3069732463195677


In [54]:
def get_preds(g): 
    x = g[(g.order_number == g.order_number.max())]
    x = x[x.reordered == 1]
    x = set(x.product_id.values)
    if len(x) == 0:
        x = set([0])
    return x

preds = prior[prior.user_id <= nusers].groupby("user_id").apply(get_preds)
preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
preds["true_pos"] = preds.apply(get_true_pos, axis=1)
preds["prec"] = preds.apply(lambda g: len(g.true_pos) / len(g.pred), axis=1)
preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
preds["f1"] = preds["f1"].fillna(0)
preds

,real,pred,true_pos,prec,recall,f1
user_id,,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{37602, 31683, 47272, 29993, 27690, 15592, 131...","{47272, 40852, 13198}",0.25,0.38,0.30
8,"{15937, 41540, 23165, 21903}","{17794, 18531, 34358, 23165, 2078}",{23165},0.20,0.25,0.22
17,"{12720, 16797, 18534}","{14146, 48933, 18534, 9006, 26767, 7350, 16797}","{16797, 18534}",0.29,0.67,0.40
18,"{25997, 22031, 21137, 36216, 47546}",{0},{},0.00,0.00,0.00
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{29223, 48205, 36784, 6352, 24852, 41273, 44570}","{24852, 48205, 29223}",0.43,0.15,0.22
...,...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}","{38928, 23915}",{38928},0.50,0.25,0.33
206156,"{42820, 20298, 33452, 34483, 15999}",{0},{},0.00,0.00,0.00
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...","{27104, 12576, 45570, 40706, 24964, 46979, 851...",{23926},0.05,0.12,0.07


In [55]:
print("resultado final:", preds.f1.mean())

resultado final: 0.3230450321701429


In [56]:
def get_preds(g, th=0.5): 
    nro_orders = g.order_id.unique().shape[0]
    x = g.product_id.value_counts() / nro_orders
    x = set(x[x >= th].index)
    if len(x) == 0:
        x = set([0])
    return x

preds = prior[prior.user_id <= nusers].groupby("user_id").apply(get_preds)
preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
preds["true_pos"] = preds.apply(get_true_pos, axis=1)
preds["prec"] = preds.apply(lambda g: len(g.true_pos) / len(g.pred), axis=1)
preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
preds["f1"] = preds["f1"].fillna(0)
preds

,real,pred,true_pos,prec,recall,f1
user_id,,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{21137, 37602, 40852, 17638}","{40852, 17638}",0.50,0.25,0.33
8,"{15937, 41540, 23165, 21903}","{17794, 18531, 21903, 14992, 9839, 34358, 2898...","{23165, 21903}",0.20,0.50,0.29
17,"{12720, 16797, 18534}","{17762, 18534, 7350}",{18534},0.33,0.33,0.33
18,"{25997, 22031, 21137, 36216, 47546}","{36216, 47546}","{36216, 47546}",1.00,0.40,0.57
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{46480, 24852, 48205}","{46480, 24852, 48205}",1.00,0.15,0.26
...,...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}",{0},{},0.00,0.00,0.00
206156,"{42820, 20298, 33452, 34483, 15999}","{28227, 42820, 32141}",{42820},0.33,0.20,0.25
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...",{0},{},0.00,0.00,0.00


In [57]:
print("resultado final:", preds.f1.mean())

resultado final: 0.2779553544261203


In [58]:
res = {}
for th in [i / 10 for i in range(11)]:
    preds = prior[prior.user_id <= nusers].groupby("user_id").apply(lambda x: get_preds(x, th))
    preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
    preds["true_pos"] = preds.apply(get_true_pos, axis=1)
    preds["prec"] = preds.apply(lambda g: len(g.true_pos) / len(g.pred), axis=1)
    preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
    preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
    preds["f1"] = preds["f1"].fillna(0)
    res[th] = preds.f1.mean()
res

{0.0: 0.21368530049232756,
 0.1: 0.26991880860938483,
 0.2: 0.3119235933482408,
 0.3: 0.3187938899785179,
 0.4: 0.30549751941403747,
 0.5: 0.2779553544261203,
 0.6: 0.23827631831798576,
 0.7: 0.19332065376211197,
 0.8: 0.15752149025734244,
 0.9: 0.1237035070810822,
 1.0: 0.11380491978794535}

In [60]:
preds

,real,pred,true_pos,prec,recall,f1
user_id,,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...",{0},{},0.00,0.00,0.00
8,"{15937, 41540, 23165, 21903}","{17794, 23165, 2078}",{23165},0.33,0.25,0.29
17,"{12720, 16797, 18534}",{0},{},0.00,0.00,0.00
18,"{25997, 22031, 21137, 36216, 47546}",{0},{},0.00,0.00,0.00
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...",{0},{},0.00,0.00,0.00
...,...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}",{0},{},0.00,0.00,0.00
206156,"{42820, 20298, 33452, 34483, 15999}",{0},{},0.00,0.00,0.00
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...",{0},{},0.00,0.00,0.00


In [63]:
prior[prior.user_id == 8].product_id.value_counts()

2078     3
23165    3
17794    3
28985    2
14992    2
        ..
42736    1
1529     1
26882    1
20920    1
11136    1
Name: product_id, dtype: int64

In [65]:
prior[prior.user_id == 8].order_id.unique()

array([ 600894, 2570360, 2979257], dtype=int64)

In [67]:
a = set([1, 2])
a.add(3)
a

{1, 2, 3}

In [68]:
def get_preds(g, th=0.5): 
    nro_orders = g.order_id.unique().shape[0]
    x = g.product_id.value_counts() / nro_orders
    x = set(x[x >= th].index)
    x.add(0)
    if len(x) == 0:
        x = set([0])
    return x

preds = prior[prior.user_id <= nusers].groupby("user_id").apply(get_preds)
preds = pd.concat([real.rename("real"), preds.rename("pred")], axis=1)
preds["true_pos"] = preds.apply(get_true_pos, axis=1)
preds["prec"] = preds.apply(lambda g: len(g.true_pos) / len(g.pred), axis=1)
preds["recall"] = preds.apply(lambda g: len(g.true_pos) / len(g["real"]), axis=1)
preds["f1"] = 2 * (preds.prec * preds.recall) / (preds.prec + preds.recall)
preds["f1"] = preds["f1"].fillna(0)
preds

,real,pred,true_pos,prec,recall,f1
user_id,,,,,,
7,"{17638, 29894, 47272, 45066, 13198, 37999, 408...","{0, 37602, 17638, 21137, 40852}","{40852, 17638}",0.40,0.25,0.31
8,"{15937, 41540, 23165, 21903}","{0, 17794, 18531, 21903, 14992, 9839, 34358, 2...","{23165, 21903}",0.18,0.50,0.27
17,"{12720, 16797, 18534}","{17762, 0, 18534, 7350}",{18534},0.25,0.33,0.29
18,"{25997, 22031, 21137, 36216, 47546}","{36216, 0, 47546}","{36216, 47546}",0.67,0.40,0.50
43,"{11777, 33027, 23909, 46981, 29223, 45066, 196...","{46480, 0, 24852, 48205}","{46480, 24852, 48205}",0.75,0.15,0.25
...,...,...,...,...,...,...
206150,"{38928, 22362, 42500, 17957}",{0},{},0.00,0.00,0.00
206156,"{42820, 20298, 33452, 34483, 15999}","{0, 28227, 42820, 32141}",{42820},0.25,0.20,0.22
206174,"{18370, 5922, 17191, 21137, 12211, 35221, 2392...",{0},{},0.00,0.00,0.00


In [69]:
print("resultado final:", preds.f1.mean())

resultado final: 0.2717552097238286


In [72]:
preds.f1.round(1).value_counts(1).sort_index()

0.00   0.34
0.10   0.04
0.20   0.13
0.30   0.12
0.40   0.12
       ... 
0.60   0.04
0.70   0.06
0.80   0.01
0.90   0.00
1.00   0.02
Name: f1, dtype: float64